<h1>Ship's Ownership Identification</h1>

In [ ]:
# Menyiapkan Dataframe yang akan dimatching
ais_check

# Import fungsi khusus dari modul 'ais' untuk membaca dan mencocokkan data AIS dan IHS
from ais import functions as af

# Membaca file ShipData.CSV terbaru dari IHS ke dalam DataFrame Spark
# Fungsi ini kemungkinan otomatis memilih versi file terbaru
ship_data = af.read_latest_ihs_table(spark, 'ShipData.CSV')

# --- Penanganan konflik nama kolom antara DataFrame AIS dan IHS ---

# Ubah nama kolom 'callsign' di DataFrame AIS menjadi 'callsign_ais'
# untuk menghindari duplikasi nama saat melakukan join atau matching
ais_check = ais_check.withColumnRenamed("callsign", "callsign_ais")

# Ubah nama kolom 'callsign' di DataFrame IHS menjadi 'callsign_ihs'
ship_data = ship_data.withColumnRenamed("callsign", "callsign_ihs")

# Ubah nama kolom 'Draught' di DataFrame AIS menjadi 'Draught_ais'
ais_check = ais_check.withColumnRenamed("Draught", "Draught_ais")

# Ubah nama kolom 'Draught' di DataFrame IHS menjadi 'Draught_ihs'
ship_data = ship_data.withColumnRenamed("Draught", "Draught_ihs")

# --- Proses pencocokan data antara AIS dan IHS ---

# Melakukan pencocokan kapal antara data AIS (ais_check) dan IHS (ship_data)
# menggunakan algoritma 'Levenshtein' untuk mengukur kesamaan string (misalnya nama kapal)
# Threshold kemiripan yang digunakan adalah 0.3
# Hasil pencocokan disimpan dalam DataFrame df_match
df_match = af.match_ais_ihs(ais_check, ship_data, 'Levenshtein', 0.3)

In [ ]:
from IPython.display import HTML
import base64  

def create_download_link(spark_df, title="Download CSV file", filename="match_1sd31_01_25_belawan.csv"):  
    # Konversi DataFrame Spark ke Pandas
    df = spark_df.toPandas()
    
    # Simpan sebagai CSV dalam bentuk teks
    csv = df.to_csv(index=False)
    
    # Encode ke base64
    b64 = base64.b64encode(csv.encode()).decode()
    
    # Buat HTML link untuk download
    html = f'<a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">{title}</a>'
    
    return HTML(html)

# Menyimpan hasil
create_download_link(df_match)